1. 연관규칙 분석을 수행하기 위해 lotto 데이터셋을 transaction 데이터로 변환하시오. 
그리고 변환된 데이터에서 가장 많이 등장한 상위 10개의 로또 번호를 막대그래프로 출력하고 이에 대해 설명하시오

In [85]:
import pandas as pd
import numpy as np

lotto = pd.read_csv("lotto.csv")
lotto.head()

,time_id,num1,num2,num3,num4,num5,num6
0,859,8,22,35,38,39,41
1,858,9,13,32,38,39,43
2,857,6,10,16,28,34,38
3,856,10,24,40,41,43,44
4,855,8,15,17,19,43,44


In [86]:
time_id = lotto[['time_id']]
lotto.drop('time_id',axis=1,inplace = True)
lotto.head()

,num1,num2,num3,num4,num5,num6
0,8,22,35,38,39,41
1,9,13,32,38,39,43
2,6,10,16,28,34,38
3,10,24,40,41,43,44
4,8,15,17,19,43,44


In [87]:
lotto.shape[0]

859

In [91]:
lotto.iloc[1,2]

32

In [100]:
num = []
for i in range(lotto.shape[0]):
    num.append(lotto.iloc[i,j] for j in range(lotto.shape[1]))
    
num

[<generator object <genexpr> at 0x000001D0B0570DD0>,
 <generator object <genexpr> at 0x000001D0B0570F20>,
 <generator object <genexpr> at 0x000001D0B0570EB0>,
 <generator object <genexpr> at 0x000001D0B0570F90>,
 <generator object <genexpr> at 0x000001D0B0570CF0>,
 <generator object <genexpr> at 0x000001D0B0574040>,
 <generator object <genexpr> at 0x000001D0B05740B0>,
 <generator object <genexpr> at 0x000001D0B0574120>,
 <generator object <genexpr> at 0x000001D0B0574190>,
 <generator object <genexpr> at 0x000001D0B0574200>,
 <generator object <genexpr> at 0x000001D0B0574270>,
 <generator object <genexpr> at 0x000001D0B05742E0>,
 <generator object <genexpr> at 0x000001D0B0574350>,
 <generator object <genexpr> at 0x000001D0B05743C0>,
 <generator object <genexpr> at 0x000001D0B0574430>,
 <generator object <genexpr> at 0x000001D0B05744A0>,
 <generator object <genexpr> at 0x000001D0B0574510>,
 <generator object <genexpr> at 0x000001D0B0574580>,
 <generator object <genexpr> at 0x000001D0B057